# Computer Vision: Image Analysis

Holding

In [2]:
!wget -O apples.png https://img.freepik.com/premium-photo/apple-fruit-basket-wooden-table-ripe-red-apples_73523-7545.jpg 

--2023-11-17 20:18:43--  https://img.freepik.com/premium-photo/apple-fruit-basket-wooden-table-ripe-red-apples_73523-7545.jpg
Resolving img.freepik.com (img.freepik.com)... 23.38.194.45, 23.38.194.40, 2600:1404:ec00:e::17da:e85a, ...
Connecting to img.freepik.com (img.freepik.com)|23.38.194.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79745 (78K) [image/jpeg]
Saving to: ‘apples.png’

apples.png          100%[===================>]  77.88K   244KB/s    in 0.3s    

2023-11-17 20:18:45 (244 KB/s) - ‘apples.png’ saved [79745/79745]



In [ ]:
import cv2
import numpy as np

# Load the image of apples
image = cv2.imread('apples.png')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(gray, (15, 15), 0)

# Perform edge detection
edges = cv2.Canny(blurred, 30, 150)

# Find contours in the edge-detected image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Initialize a counter for the number of apples
apple_count = 0

# Iterate through the contours and filter out small ones (potential noise)
for contour in contours:
    if cv2.contourArea(contour) > 1000:
        apple_count += 1

# Print the number of apples found
print(f"Number of apples: {apple_count}")

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2

# Load and preprocess the image of apples
image = cv2.imread('apples.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB format
image = cv2.resize(image, (32, 32))  # Resize to match CIFAR-10 input size

# Create a model to classify CIFAR-10 classes
model = keras.applications.VGG16(weights='imagenet', include_top=True)

# Classify the image using the pre-trained model
predictions = model.predict(image.reshape(1, 32, 32, 3))

# Get the class label with the highest probability
predicted_class = np.argmax(predictions)

# Print the predicted class label (CIFAR-10 class)
print(f"Predicted class (CIFAR-10): {predicted_class}")